In [ ]:
import os
cd = os.getcwd()
print(cd)

from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
import numpy as np

#Train-test split

In [ ]:
# load processed target labels and static features data
path = '/content/gdrive/My Drive/Colab Notebooks/SHP6004/TeamShare/ProcessedData/'
flat_pre = pd.read_csv(path + 'preprocessed_staticFeatures_SB.csv', index_col='patient')
labels_pre = pd.read_csv(path + 'preprocessed_labels_SB.csv', index_col='patient')

In [ ]:
# load processed timeseries data
path = '/content/gdrive/My Drive/Colab Notebooks/SPH6004/TeamShare/ProcessedData/'
timeseries_pre = pd.read_csv(path + '7 Dynamic_Final.csv',index_col='patient')
timeseries_pre.head()

,time_slot,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium,hematocrit,hemoglobin,mch,mchc,mcv,platelet,rbc,rdw,wbc
patient,,,,,,,,,,,,,,,,,,,
20001305,1,-0.120654,0.117566,0.470452,3.585920,0.639491,-0.605244,0.039963,0.692532,-0.020649,0.010569,0.019966,0.004856,0.021439,-0.006427,0.059107,0.030855,0.144165,-0.157610
20001305,2,-0.534140,0.499282,0.508936,2.901384,0.501881,-0.556630,-0.023739,0.345614,0.504043,0.005910,0.014868,0.005020,0.023229,-0.007265,0.108619,0.021164,0.094051,-0.158146
20001305,3,-0.534140,0.499282,0.508936,2.901384,0.501881,-0.556630,-0.023739,0.345614,0.504043,0.005910,0.014868,0.005020,0.023229,-0.007265,0.108619,0.021164,0.094051,-0.158146
20001305,4,-0.534140,0.308424,0.585904,2.901384,0.639491,-0.556630,-0.253067,0.519073,-0.282995,-0.005319,0.006977,0.011655,0.034670,-0.009290,0.037785,0.012137,0.134085,-0.186103
20001361,1,-0.120654,-0.455009,-0.106806,-2.004458,0.639491,0.221204,-0.342250,-0.174763,1.815772,0.760737,0.961336,0.144109,0.752628,-0.341379,0.245161,0.819425,-1.093515,0.419186


In [ ]:
# load notes data
import pickle

path = '/content/gdrive/My Drive/Colab Notebooks/SPH6004/TeamShare/ProcessedData/text_dataframe.pkl'
with open(path, 'rb') as f:
    notes_data = pickle.load(f)

notes_data.rename(columns={'id': 'patient'}, inplace=True)
notes_data.set_index('patient', inplace=True)

path = '/content/gdrive/My Drive/Colab Notebooks/SPH6004/TeamShare/ProcessedData/TF-IDF.pkl'
with open(path, 'rb') as f:
    notes_tfid = pickle.load(f)


## Filter patients based on timeseries data

In [ ]:
# filter out any patients that don't have timeseries and are removed from labels
ts_patients = timeseries_pre.index.unique().intersection(labels_pre.index)
flat_pre = flat_pre.loc[ts_patients].copy()
labels_pre = labels_pre.loc[ts_patients].copy()

In [ ]:
#check shape
print('Shapes:',flat_pre.shape,labels_pre.shape)

Shapes: (17255, 31) (17255, 2)


In [ ]:
sel = np.where(notes_data.index.isin(ts_patients))[0]
notes_tfid = notes_tfid[sel].copy()
notes_data = notes_data.loc[ts_patients].copy()

In [ ]:
#check shape
print('Shapes:',len(notes_tfid), notes_data.shape, len(notes_data.index.unique()) )

Shapes: 17255 (17255, 1) 17255


In [ ]:
tfid_df = pd.DataFrame(notes_tfid, index=notes_data.index)
tfid_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
patient,,,,,,,,,,,,,,,,,,,,,
20001305,0.206751,0.000000,0.000000,0.070021,0.000000,0.037730,0.210854,0.030022,0.000000,0.00000,...,0.000000,0.151338,0.040729,0.071079,0.098968,0.038111,0.116614,0.035512,0.000000,0.040249
20001361,0.123598,0.000000,0.000000,0.026910,0.031836,0.086999,0.126051,0.023075,0.150289,0.00000,...,0.000000,0.000000,0.000000,0.054632,0.025356,0.000000,0.143410,0.109180,0.027600,0.000000
20001770,0.250956,0.041097,0.095795,0.104308,0.041135,0.000000,0.186135,0.029815,0.072819,0.00000,...,0.037577,0.100197,0.040449,0.000000,0.032762,0.000000,0.231622,0.070535,0.035662,0.039971
20002506,0.137084,0.000000,0.000000,0.064283,0.038026,0.000000,0.225837,0.000000,0.168289,0.15184,...,0.000000,0.023156,0.018696,0.048940,0.075715,0.052481,0.171293,0.065204,0.164831,0.036950
20003425,0.139454,0.000000,0.000000,0.000000,0.055875,0.025449,0.221232,0.081000,0.065943,0.00000,...,0.127608,0.000000,0.027472,0.047942,0.066753,0.025706,0.125850,0.047905,0.000000,0.081443


## Create split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#creates a folder
def create_folder(parent_path, folder):
    if not parent_path.endswith('/'):
        parent_path += '/'
    folder_path = parent_path + folder
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    return folder_path

#shuffles the patients
def shuffle_stays(stays, seed=9):
    return shuffle(stays, random_state=seed)

#saves the selected patients
def process_table(table_name, table, stays, folder_path):
    table = table.loc[stays].copy()
    table.to_csv('{}/{}.csv'.format(folder_path, table_name))
    return

In [ ]:
# split data into train, test and validtion
def split_train_test(label_data,flat_data,time_data,notes_data, save_path,seed=9):

    # we split by unique patient identifier to make sure there are no patients
    # that cross into both the train and the test sets
    patients = label_data.index.unique(level=0)

    train, test = train_test_split(patients, test_size=0.15, random_state=seed)
    train, val = train_test_split(train, test_size=0.15/0.85, random_state=seed)

    for partition_name, partition in zip(['train', 'val', 'test'], [train, val, test]):
        print('==> Preparing {} data...'.format(partition_name))
        stays = label_data.loc[label_data.index.isin(partition)].index
        folder_path = create_folder(save_path, partition_name)

        #stays.txt will contain the patient ids for the particular set of data
        with open(folder_path + '/stays.txt', 'w') as f:
            for stay in stays:
                f.write("%s\n" % stay)
        stays = shuffle_stays(stays, seed=9)

        for table_name, table in zip(['labels', 'flat', 'timeseries','tfid_dataframe'],
                                      [label_data, flat_data, time_data, notes_data]):
            process_table(table_name, table, stays, folder_path)


    return

In [ ]:
save_path = '/content/gdrive/My Drive/Colab Notebooks/SPH6004/TeamShare/ProcessedData/'
split_train_test(labels_pre,flat_pre,timeseries_pre,tfid_df,save_path)

==> Preparing train data...
==> Preparing val data...
==> Preparing test data...
